In [2]:
import numpy as np
import pandas as pd

In [ ]:
# idea: record LHS for several good and bad runs and plot on one graph to determine good vs bad criterion for each experiment!

# EXP (no noise)

In [3]:
results_dir = 'C:/Users/Blake Bullwinkel/Documents/Harvard/denn/denn/'

rand_reps = pd.read_csv(results_dir+'rand_reps.csv')
rand_reps_mse = pd.read_csv(results_dir+'rand_reps_mse.csv')
rand_reps_lhs = pd.read_csv(results_dir+'rand_reps_lhs.csv')

In [4]:
rand_reps

,Unnamed: 0,seed,g_lr,d_lr,mean_squared_error
0,0,0,0.01,0.01,3.543404e-07
1,1,1,0.01,0.01,1.149714e-06
2,2,2,0.01,0.01,4.309146e-07
3,3,3,0.01,0.01,2.842818e-07
4,4,4,0.01,0.01,3.810515e-07
5,5,5,0.01,0.01,6.731857e-06
6,6,6,0.01,0.01,3.730272e-06
7,7,7,0.01,0.01,1.330974e-07
8,8,8,0.01,0.01,4.961690e-07
9,9,9,0.01,0.01,1.677736e-06


In [5]:
rand_reps_mse

,Unnamed: 0,0_0.01_0.01,1_0.01_0.01,2_0.01_0.01,3_0.01_0.01,4_0.01_0.01,5_0.01_0.01,6_0.01_0.01,7_0.01_0.01,8_0.01_0.01,9_0.01_0.01
0,0,2.953083e-02,0.029352,2.951897e-02,2.962921e-02,2.948742e-02,0.029593,0.029557,2.948728e-02,3.021730e-02,0.029527
1,1,5.264159e-01,0.527372,5.266660e-01,5.258812e-01,5.269651e-01,0.526283,0.525310,5.269603e-01,5.210621e-01,0.526570
2,2,2.261983e+00,2.264005,2.263057e+00,2.260728e+00,2.263235e+00,2.261693,2.258324,2.263068e+00,2.248367e+00,2.262653
3,3,4.362053e+00,4.364576,4.363843e+00,4.359896e+00,4.365496e+00,4.363466,4.359523,4.364073e+00,4.349565e+00,4.363213
4,4,6.323383e+00,6.321803,6.320223e+00,6.319757e+00,6.325071e+00,6.325830,6.317596,6.322909e+00,6.317085e+00,6.321085
...,...,...,...,...,...,...,...,...,...,...,...
995,995,3.543409e-07,0.000001,4.309181e-07,2.842853e-07,3.810511e-07,0.000007,0.000004,1.330974e-07,4.961712e-07,0.000002
996,996,3.543409e-07,0.000001,4.309153e-07,2.842853e-07,3.810511e-07,0.000007,0.000004,1.330976e-07,4.961712e-07,0.000002
997,997,3.543404e-07,0.000001,4.309153e-07,2.842827e-07,3.810511e-07,0.000007,0.000004,1.330976e-07,4.961712e-07,0.000002
998,998,3.543404e-07,0.000001,4.309137e-07,2.842824e-07,3.810515e-07,0.000007,0.000004,1.330974e-07,4.961690e-07,0.000002


In [6]:
rand_reps_lhs

,Unnamed: 0,0_0.01_0.01,1_0.01_0.01,2_0.01_0.01,3_0.01_0.01,4_0.01_0.01,5_0.01_0.01,6_0.01_0.01,7_0.01_0.01,8_0.01_0.01,9_0.01_0.01
0,0,1.286383,1.285403,1.286000,1.287139,1.286422,1.286662,1.286794,1.286682,1.288441,1.286958
1,1,0.186444,0.185534,0.185428,0.187120,0.187046,0.186910,0.186640,0.186386,0.189907,0.185795
2,2,0.777511,0.776802,0.780713,0.777752,0.777041,0.778900,0.778283,0.783300,0.777651,0.781533
3,3,1.614042,1.611710,1.615977,1.613197,1.616503,1.611613,1.612657,1.615913,1.608484,1.609182
4,4,2.249370,2.259491,2.253018,2.250260,2.257396,2.250813,2.257128,2.251454,2.251895,2.251659
...,...,...,...,...,...,...,...,...,...,...,...
995,995,0.000697,0.001431,0.000742,0.000727,0.000765,0.002716,0.002388,0.000436,0.000757,0.001519
996,996,0.000692,0.001452,0.000709,0.000688,0.000757,0.002710,0.002407,0.000455,0.000736,0.001522
997,997,0.000745,0.001445,0.000752,0.000712,0.000754,0.002716,0.002385,0.000457,0.000741,0.001529
998,998,0.000708,0.001453,0.000839,0.000688,0.000750,0.002704,0.002406,0.000460,0.000742,0.001521
